In [ ]:
from google.colab import files

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#!pip install scikit-survival
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
from google.colab import output
output.enable_custom_widget_manager()

import catboost
from catboost import CatBoostRegressor, Pool

print(catboost.__version__)

In [ ]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c house-prices-advanced-regression-techniques
!unzip -q "house-prices-advanced-regression-techniques.zip" -d /content/house_prices_advanced_regression_techniques

100% 199k/199k [00:00<00:00, 413kB/s]
100% 199k/199k [00:00<00:00, 412kB/s]


In [ ]:
data = pd.read_csv('/content/house_prices_advanced_regression_techniques/train.csv', index_col=0)
data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [ ]:
#data.info()

In [ ]:
data.MasVnrType.fillna('None', inplace=True)
data.BsmtQual.fillna('None', inplace=True)
data.BsmtCond.fillna('None', inplace=True)
data.BsmtExposure.fillna('None', inplace=True)
data.BsmtFinType1.fillna('None', inplace=True)
data.BsmtFinType2.fillna('None', inplace=True)
data.Electrical.fillna('None', inplace=True)
data.FireplaceQu.fillna('None', inplace=True)
data.GarageType.fillna('None', inplace=True)
data.GarageFinish.fillna('None', inplace=True)
data.GarageQual.fillna('None', inplace=True)
data.GarageCond.fillna('None', inplace=True)

In [ ]:
data = data.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
#data.info()

In [ ]:
random_seed = 0
x_train, x_val = train_test_split(data, 
                                  test_size=0.1, 
                                  random_state=random_seed)

print(x_train.shape)
print(x_val.shape)

print(x_train.head())

(1314, 76)
(146, 76)
      MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
Id                                                                            
931           20       RL         73.0     8925   Pave      IR1         HLS   
657           20       RL         72.0    10007   Pave      IR1         Lvl   
46           120       RL         61.0     7658   Pave      Reg         Lvl   
1349          20       RL          NaN    16196   Pave      IR3         Low   
56            20       RL        100.0    10175   Pave      IR1         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPorch 3SsnPorch ScreenPorch  \
Id                                  ...                                       
931     AllPub    Inside       Gtl  ...             0         0           0   
657     AllPub    Inside       Gtl  ...             0         0           0   
46      AllPub    Inside       Gtl  ...             0         0           0   
1349    AllPub    Inside      

In [ ]:
features = list(data.columns)
features.remove('SalePrice')
print(features)

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'Y

In [ ]:
cat_features = ['MSZoning', 'Street', 'LotShape', 'LandContour',
                'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                'Condition1', 'Condition2','BldgType', 'HouseStyle', 'RoofStyle',
                'RoofMatl', 'Exterior1st', 'Exterior2nd','MasVnrType', 'ExterQual',
                'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
                'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
                'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType',
                'SaleCondition']

train_pool = Pool(data=x_train[features], label=x_train['SalePrice'], cat_features=cat_features)
test_pool = Pool(data=x_val[features], label=x_val['SalePrice'], cat_features=cat_features)      

In [ ]:
model = CatBoostRegressor(l2_leaf_reg=0.0005,
                          loss_function='RMSE',                                                                       
                          eval_metric='MAE',                                                    
                          metric_period=50,   
                          grow_policy='Depthwise',
                         )

In [ ]:
model.fit(train_pool, 
          eval_set=test_pool, 
          use_best_model=True, 
          early_stopping_rounds=50,
          verbose=True,
          )

0:	learn: 56364.6777947	test: 53469.8972763	best: 53469.8972763 (0)	total: 81.4ms	remaining: 1m 21s
50:	learn: 21245.9471032	test: 21810.0059628	best: 21810.0059628 (50)	total: 1.26s	remaining: 23.4s
100:	learn: 13498.3165759	test: 17171.3023632	best: 17171.3023632 (100)	total: 2.52s	remaining: 22.5s
150:	learn: 11041.4244818	test: 16487.9799995	best: 16487.9799995 (150)	total: 3.7s	remaining: 20.8s
200:	learn: 9864.8194284	test: 16443.9456736	best: 16419.9856549 (185)	total: 4.8s	remaining: 19.1s
250:	learn: 9043.4526015	test: 16419.4876085	best: 16414.9084494 (223)	total: 5.91s	remaining: 17.6s
300:	learn: 8372.3556880	test: 16348.5212022	best: 16334.1743892 (297)	total: 6.96s	remaining: 16.2s
350:	learn: 7737.5669346	test: 16279.4661009	best: 16279.4661009 (350)	total: 8.07s	remaining: 14.9s
400:	learn: 7094.6407901	test: 16282.3477192	best: 16268.8993282 (385)	total: 9.21s	remaining: 13.8s
450:	learn: 6594.7551639	test: 16314.2293864	best: 16254.5783254 (411)	total: 10.3s	remaining

Public score 0.

In [ ]:
test_data = pd.read_csv('/content/house_prices_advanced_regression_techniques/test.csv', index_col=0)
test_data = test_data.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   LotShape       1459 non-null   object 
 6   LandContour    1459 non-null   object 
 7   Utilities      1457 non-null   object 
 8   LotConfig      1459 non-null   object 
 9   LandSlope      1459 non-null   object 
 10  Neighborhood   1459 non-null   object 
 11  Condition1     1459 non-null   object 
 12  Condition2     1459 non-null   object 
 13  BldgType       1459 non-null   object 
 14  HouseStyle     1459 non-null   object 
 15  OverallQual    1459 non-null   int64  
 16  OverallCond    1459 non-null   int64  
 17  YearBuilt      1459 non-null   int64  
 18  YearR

In [ ]:
for i in cat_features:
    test_data[i].fillna('None', inplace=True)

In [ ]:
predict = model.predict(test_data)

In [ ]:
sample_submission = pd.read_csv('/content/house_prices_advanced_regression_techniques/sample_submission.csv')
sample_submission['SalePrice'] = predict
sample_submission.to_csv('submission.csv', index=False)
sample_submission

,Id,SalePrice
0,1461,118069.970024
1,1462,157601.252955
2,1463,187071.958069
3,1464,189766.448432
4,1465,194612.103614
...,...,...
1454,2915,83156.679079
1455,2916,82850.202156
1456,2917,161207.545985
1457,2918,117701.970093


In [ ]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Submition from Colab"

100% 33.7k/33.7k [00:02<00:00, 13.5kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques